In [3]:
# #we will import a bunch of stuff, and become a senior ML engineer

import numpy as np
import h5py
import matplotlib.pyplot as plt
import copy
import sklearn
import sklearn.datasets
import sklearn.linear_model

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
class Activ:

    def __init__(self,p):

        def relu (x):
            if isinstance(x, np.ndarray):
                return np.where(x > 0, x, 0.0)
            else:
                if x > 0:
                    return x
                else:
                    return 0.0

        def relu_grad (x):
            if isinstance(x, np.ndarray):
                return np.where(x > 0, 1.0, 0.0)
            else:
                if x > 0:
                    return 1.0
                else:
                    return 0.0

        relu_dict = {}
        relu_dict["activ"] = relu
        relu_dict["grad"] = relu_grad

        def binStep (x):
           if isinstance(x, np.ndarray):
               return np.where(x > 0, 1.0, 0.0)
           else:
                if x > 0:
                    return 1.0
                else:
                    return 0.0

        def binStep_grad (x):
            return binStep(x)

        binStep_dict = {}
        binStep_dict["activ"] = binStep
        binStep_dict["grad"] = binStep_grad

        def sigmoid (x):
            return (1/(1+np.exp(-x)))

        def sigmoid_grad (x):
            return sigmoid(x) * (1 - sigmoid(x))

        sigmoid_dict = {}
        sigmoid_dict["activ"] = sigmoid
        sigmoid_dict["grad"] = sigmoid_grad


        def tanh (x):
            return ((2/(1 + np.exp(-2 * x))) - 1)
        def tanh_grad (x):
            return (1 - (tanh(x)**2))

        tanh_dict = {}
        tanh_dict["activ"] = tanh
        tanh_dict["grad"] = tanh_grad

        def arctan (x):
                return np.arctan(x)
        def arctan_grad (x):
                return (1/(1+(x**2)))

        arctan_dict = {}
        arctan_dict["activ"] = arctan
        arctan_dict["grad"] = arctan_grad

        def Parctan (x): 
            q = (2 * p)/(np.pi)
            return q * ( np.arctan(x) + (np.pi/2))
        def Parctan_grad (x):
            q = (2 * p)/(np.pi)
            return (q * (1/(1+(x**2))))
            
        Parctan_dict = {}
        Parctan_dict["activ"] = Parctan
        Parctan_dict["grad"] = Parctan_grad

        def prelu (x):
            if isinstance(x, np.ndarray):
                return np.where(x > 0, x, p * x)
            else:
                if x > 0:
                    return x
                else:
                    return p * x

        def prelu_grad (x):
            if isinstance(x, np.ndarray):
                return np.where(x > 0, 1.0, p)
            else:
                if x > 0:
                    return 1
                else:
                    return p

        prelu_dict = {}
        prelu_dict["activ"] = prelu
        prelu_dict["grad"] = prelu_grad

        def elu (x):
            if isinstance(x, np.ndarray):
                return np.where(x > 0, x, (p * (np.exp(x) - 1)))
            else:
                if x > 0:
                    return x
                else:
                    return (p * (np.exp(x) - 1))

        def elu_grad (x):
            if isinstance(x, np.ndarray):
                return np.where(x > 0, 1.0, elu(x) + p)
            else:
                if x > 0:
                    return 1
                else:
                    return (elu(x) + p)

        elu_dict = {}
        elu_dict["activ"] = elu
        elu_dict["grad"] = elu_grad

        def softplus (x):
            return (np.log(1 + np.exp(x)))

        def softplus_grad (x):
            return sigmoid(x)

        softplus_dict = {}
        softplus_dict["activ"] = softplus
        softplus_dict["grad"] = softplus_grad

        
        Activators = {}
        Activators["relu"] = relu_dict
        Activators["binStep"] = binStep_dict
        Activators["sigmoid"] = sigmoid_dict
        Activators["tanh"] = tanh_dict
        Activators["arctan"] = arctan_dict
        Activators["prelu"] = prelu_dict
        Activators["elu"] = elu_dict
        Activators["softplus"] = softplus_dict
        Activators["Parctan"] = Parctan_dict
        self.Activ_choices = Activators

In [5]:
class LossFuncs:
    def __init__ (self,p):

        def L2 (y_o, y):
            return ((y_o - y)**2)
        def L2_grad(y_o, y):
            return (2 * (y_o - y))

        L2_dict = {}
        L2_dict["activ"] = L2
        L2_dict["grad"] = L2_grad

        self.e = 0.0001
        def crossENT (y_o, y):
            return -((y * np.log(y_o)) + ((1-y) * (np.log(1-y_o))))
        def crossENT_grad (y_o, y):
            return -(np.divide(y, y_o + self.e) - np.divide(1-y, (1-y_o) + self.e))

        crossENT_dict = {}
        crossENT_dict["activ"] = crossENT
        crossENT_dict["grad"] = crossENT_grad


        def L1 (y_o, y):
            return np.abs(y_o - y)
        def L1_grad (y_o, y):
            return np.where(y_o - y > 0, 1, -1)

        L1_dict = {}
        L1_dict["activ"] = L1
        L1_dict["grad"] = L1_grad


        def Huber (y_o, y):
            z = np.where(L1 (y_o,y) > p, (p * L1(y_o,y)) - (0.5 * (p ** 2)), L1 (y_o,y))
            return (z * z)/2
        def Huber_grad (y_o, y):
            z = np.where(L1 (y_o,y) > p, (p * L1 (y_o,y)) - (0.5 * (p ** 2)), L1 (y_o,y))
            dz = np.where(L1 (y_o,y) > p, p * L1_grad (y_o,y), L1_grad (y_o,y))
            return z * dz

        Huber_dict = {}
        Huber_dict["activ"] = Huber
        Huber_dict["grad"] = Huber_grad

        def hinge (y_o, y):
            return np.maximum(0, (1 - (y * y_o)))
        def hinge_grad (y_o, y):
            return np.where((1-(y * y_o)) > 0, -1, 0.0)

        hinge_dict = {}
        hinge_dict["activ"] = hinge
        hinge_dict["grad"] = hinge_grad

        Losses = {}
        Losses["L2"] = L2_dict
        Losses["L1"] = L1_dict
        Losses["crossENT"] = crossENT_dict
        Losses["Huber"] = Huber_dict
        Losses["hinge"] = hinge_dict

        self.Loss_choices = Losses

In [6]:
class DeepNetwork:
    def __init__ (self, X, Y, W_LB, W_UB, HL_dims, p_activ, p_loss, activ_choice, fin_activ_choice, loss_choice, class_markers):

        ACs = set(["relu", "binStep", "sigmoid", "tanh", "arctan", "prelu", "elu", "softplus", "Parctan"])
        LCs = set(["L1", "L2", "crossENT", "Huber", "hinge"])

        if (activ_choice not in ACs) or (fin_activ_choice not in ACs) or (loss_choice not in LCs):

            raise ValueError("Please enter a valid activator string: relu, binStep, sigmoid, tanh, arctan, prelu, elu, softplus, Parctan")
            raise ValueError("Please enter a valid loss string: L2, L1, crossENT, Huber, hinge")
            
        else:
            activ_dict = Activ(p_activ)
            self.activator = activ_dict.Activ_choices[activ_choice]
            self.fin_activator = activ_dict.Activ_choices[fin_activ_choice]

            loss_dict = LossFuncs(p_loss)
            self.losser = loss_dict.Loss_choices[loss_choice]

            HL_dims.insert(0, X.shape[0])
            HL_dims.append(Y.shape[0])
            self.n = len(HL_dims)

            self.weights_array = [None]*self.n
            self.weights_array[0] = -1.0

            self.bias_array = [None]*self.n
            self.bias_array[0] = -1.0

            for i in range (1,self.n):
                self.weights_array[i] = np.random.uniform(W_LB, W_UB, (HL_dims[i],HL_dims[i-1]))
                self.bias_array[i] = np.zeros((HL_dims[i],1))

            self.Z_array = [None]*self.n
            self.Z_array[0] = -1.0

            self.Activ_array = [None]*self.n
            self.Activ_array [0] = X

            self.dW_array = [None]*self.n
            self.dW_array[0] = -1.0

            self.dZ_array = [None]*self.n
            self.dZ_array[0] = -1.0

            self.dA_array = [None]*self.n
            self.dA_array[0] = -1.0

            self.dB_array = [None]*self.n
            self.dB_array[0] = -1.0

            self.m = X.shape[1]
            self.class_markers = class_markers
            

    def compute_Z (self,A_prev, W, B):
        return (np.dot(W,A_prev) + B)

    def forward_propogation (self,X):
        self.Activ_array [0] = X
        for i in range (1, self.n - 1):
            self.Z_array[i] = self.compute_Z (self.Activ_array[i-1], self.weights_array[i], self.bias_array[i])
            self.Activ_array[i] = self.activator["activ"](self.Z_array[i])

        self.Z_array[self.n - 1] = self.compute_Z(self.Activ_array[self.n - 2], self.weights_array[self.n - 1], self.bias_array[self.n - 1])
        self.Activ_array[self.n - 1] = self.fin_activator["activ"](self.Z_array[self.n - 1])



    def computeGRADS (self,A_prev, W, B, dZ, m) :
        dW = np.dot(dZ, A_prev.T)/m
        dB = np.sum(dZ, axis = 1, keepdims = True)/m
        dA_prev = np.dot(W.T, dZ)
        grads = {}
        grads["dW"] = dW
        grads["dB"] = dB
        grads["dA_prev"] = dA_prev
        return grads

    def backward_propogation (self,Y):
        self.dA_array[self.n - 1] = self.losser["grad"](self.Activ_array[self.n - 1],Y)
        self.dZ_array[self.n - 1] = self.fin_activator["grad"](self.Z_array[self.n - 1]) * self.dA_array[self.n - 1]
        #in 1 run of the loop, we are given dZ[i], we compute: dA[i-1], dB[i], dW[i].
        #then using dA[i-1] we also compute dZ[i-1], fully prepared for the next run of the loop.
        for i in reversed(range(1,self.n)):
            temp_grads = self.computeGRADS (self.Activ_array[i-1], self.weights_array[i], self.bias_array[i], self.dZ_array[i], self.m)
            self.dW_array[i] = temp_grads["dW"]
            self.dB_array[i] = temp_grads["dB"]
            if (i-1 == 0):
                break
            else:
                self.dA_array[i-1] = temp_grads["dA_prev"]
                self.dZ_array[i-1] = self.activator["grad"](self.Z_array[i-1]) * self.dA_array[i-1]

    def train_network (self,X,Y, num_iter, lrn_rate):
        for i in range (0, num_iter):
            self.forward_propogation(X)
            self.backward_propogation(Y)
            for j in range (1, self.n):
                self.weights_array[j] -= lrn_rate * self.dW_array[j]
                self.bias_array[j] -= lrn_rate * self.dB_array[j]
                
    
    
    def classif_predict (self, X_V):
        self.forward_propogation(X_V)
        array = self.Activ_array[self.n-1]
        cmark = np.array(self.class_markers, dtype=int)
        diffs = np.diff(cmark)
        cumsum_half_diffs = np.cumsum(diffs / 2)
        bin_edges = cmark[0] + cumsum_half_diffs
        classes = cmark[np.round(np.searchsorted(bin_edges, array)).astype(int)] 
        return classes

    def reg_predict (self, X_V):
        self.forward_propogation(X_V)
        return self.Activ_array[self.n - 1]

    def get_train_methods(self):
        print ("__init__ (self, X, Y, W_LB, W_UB, HL_dims, p_activ, p_loss, activ_choice, fin_activ_choice, loss_choice)")
        print("train_network (self,X,Y, num_iter, lrn_rate)")
        print("lassif_predict (self, X_V, class_markers")
        print("reg_predict (self, X_V)")

    def get_parameters (self): 
        return self.weights_array, self.bias_array, HL_dims, p_activ, p_loss, 
        
    
        
        

In [7]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
data = cancer.data
target_ = cancer.target
target = target_.reshape(target_.shape[0],1)

from sklearn.utils import shuffle
X,Y = shuffle(data, target) 

X_prev = X.T
Y_prev = Y.T
m = int(X_prev.shape[1]/3)

def split_data(X_prev, Y_prev, m):

    num_cols = X_prev.shape[1]  # Get the total number of columns
    if num_cols < 3 * m:
        raise ValueError("Number of columns must be at least 3 times the split size.")

    # Calculate the starting indices for each split
    start_train = 0
    start_val = start_train + m
    start_test = start_val + m

    # Split the matrices
    X_train = X_prev[:, start_train:start_val]
    X_val = X_prev[:, start_val:start_test]
    X_test = X_prev[:, start_test:]
    Y_train = Y_prev[:, start_train:start_val]
    Y_val = Y_prev[:, start_val:start_test]
    Y_test = Y_prev[:, start_test:]

    return X_train, X_val, X_test, Y_train, Y_val, Y_test

X_train, X_val, X_test_P, Y_train, Y_val, Y_test_P = split_data(X_prev, Y_prev, m)

X_test = X_test_P[:, 0:189]
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

Y_test = Y_test_P[:, 0:189]

print(Y_train.shape)
print(Y_val.shape)
print(Y_test.shape)


(30, 189)
(30, 189)
(30, 189)
(1, 189)
(1, 189)
(1, 189)


In [9]:
# def __init__ (self, X, Y, W_LB, W_UB, HL_dims, p_activ, p_loss, activ_choice, fin_activ_choice, loss_choice, class_markers):
class_markers = [0, 1]
t = X_train.shape[0]
HL_dims = [t, t, t-1, 2*t, t]
W_LB = -0.5
W_UB = 0.5

cancer_classifier = DeepNetwork(X_train, Y_train, W_LB, W_UB, 0.1, 0.1, HL_dims, 'relu', 'sigmoid', 'crossENT', class_markers)

AttributeError: 'float' object has no attribute 'insert'